<a href="https://colab.research.google.com/github/ansh-pachauri/GenAI-project-on-collab/blob/main/Copy_of_whisper_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers
!pip install -q langchain-community faiss-cpu langchain-google-genai
from datasets import load_dataset
!pip install torch torchaudio torchcodec

## Local Inference on GPU
Model page: https://huggingface.co/openai/whisper-small

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/openai/whisper-small)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

processor = AutoProcessor.from_pretrained("openai/whisper-small")
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-small")

In [ ]:
from datasets import Audio
import librosa

In [ ]:
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

In [ ]:
ds = ds.cast_column("audio", Audio(decode=False))

In [ ]:
sample = ds["audio"]

In [ ]:
print(sample)

In [ ]:
transcriptions = []
for i in range(2):
    sample = ds[i]["audio"]
    audio_bytes = sample["bytes"]

    # Save the audio to a temporary file
    with open(f"temp_{i}.flac", "wb") as f:
        f.write(audio_bytes)


In [ ]:
array, sr = librosa.load(f"temp_{i}.flac", sr=16000)
print(array.shape, sr)

In [ ]:
# Process and transcribe
input_features = processor(array, sampling_rate=sr, return_tensors="pt").input_features

In [ ]:
predicted_ids = model.generate(input_features)

In [ ]:
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

transcriptions.append(transcription[0]) # Append the first element as batch_decode returns a list

print(transcriptions)